<a href="https://colab.research.google.com/github/AKOKEMKada/gee/blob/main/1_centroide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install geopandas shapely pandas fiona pyproj

In [ ]:
import geopandas as gpd
import pandas as pd


# Charger les shapefiles avec Geopandas
Utilisez Geopandas pour lire votre shapefile contenant les limites administratives (par exemple, les frontières de pays, regions, provinces et communes).

In [ ]:


# Charger le shapefile
shapefile_path = "C:\\Users\\user\\Desktop\\Rapport memoire\\imple\\admin\\bfa_admin0.shp"
gdf = gpd.read_file(shapefile_path)

# Vérifier les premières lignes du DataFrame géospatial
# print(gdf.head())


In [ ]:
# list(gdf.columns)

In [ ]:

data_pcode = pd.DataFrame()
data_pcode['pcode']=gdf.iloc[:, 4]
data_pcode['pcode']=data_pcode['pcode'].str.extract(r'(\d+)', expand=False)
data_pcode

,pcode
0,None


# Calculer les centroides
Une fois les données chargées, vous pouvez facilement calculer les centroides de chaque entité géographique (polygone) dans votre shapefile en utilisant la fonction centroid de Shapely (qui est intégré dans Geopandas).
La colonne 'centroid' contiendra les coordonnées géographiques des centroides.

In [ ]:
# Calculer les centroides
gdf['centroid'] = gdf.geometry.centroid

# Vérifier les centroides calculés
print(gdf[['geometry', 'centroid']].head())


                                            geometry  \
0  POLYGON ((-0.57615 15.08258, -0.56666 15.08252...   

                    centroid  
0  POINT (-1.73479 12.27366)  


C:\Users\user\AppData\Local\Temp\ipykernel_924\1801924121.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


# Extraire les coordonnées des centroides
Extraction des coordonnées des centroides sous forme de colonnes séparées (latitude et longitude)

In [ ]:
# Extraire les coordonnées des centroides (longitude, latitude)
gdf['longitude'] = gdf['centroid'].x
gdf['latitude'] = gdf['centroid'].y

# Vérifier les coordonnées extraites
print(gdf[['longitude', 'latitude']].head())

# concatener les colonnes
result = pd.concat([gdf[['longitude', 'latitude']], data_pcode['pcode']], axis=1)
result

   longitude   latitude
0  -1.734786  12.273661


,longitude,latitude,pcode
0,-1.734786,12.273661,None


In [ ]:
# Renommer les colonnes
result = result.rename(columns={
    'longitude': 'lon',
    'latitude': 'lat',
    'pcode': 'geocode'
})

# Ajouter une colonne id en première position
result.insert(0, 'id', range(1, len(result) + 1))
result

,id,lon,lat,geocode
0,1,-1.734786,12.273661,None


# Exporter les résultats

In [ ]:
# Sauvegarder les résultats dans un fichier CSV
result[['id','lon', 'lat','geocode']].to_csv('centroide0.csv', index=False)

In [ ]:
import rasterio

# Spécifiez le chemin du fichier GeoTIFF
tif_file_path = "C:/Users/user/Desktop/Rapport memoire/imple/data_densite/bfa_pop_2022_CN_100m_R2025A_v1.tif"

# Ouvrir le fichier GeoTIFF
with rasterio.open(tif_file_path) as src:
    # Afficher des informations sur le fichier
    print(f"CRS (Coordinate Reference System): {src.crs}")
    print(f"Dimensions (rows, cols): {src.height} x {src.width}")
    print(f"Nombre de bandes: {src.count}")
    print(f"Transformée affine: {src.transform}")

    # Lire les données de la première bande (par exemple)
    band1 = src.read(1)  # Lire la première bande

    # Afficher les statistiques de la première bande
    print(f"Min de la bande 1 : {band1.min()}")
    print(f"Max de la bande 1 : {band1.max()}")
    print(f"Type de données des pixels : {band1.dtype}")

    # Afficher un extrait des valeurs (par exemple, une sous-section)
    print(f"Extrait des premières valeurs de la bande 1 :")
    print(band1[:5, :5])  # Affiche une petite section de la première bande


CRS (Coordinate Reference System): EPSG:4326
Dimensions (rows, cols): 6807 x 9501
Nombre de bandes: 1
Transformée affine: | 0.00, 0.00,-5.51|
| 0.00,-0.00, 15.08|
| 0.00, 0.00, 1.00|
Min de la bande 1 : -99999.0
Max de la bande 1 : 185.82395935058594
Type de données des pixels : float32
Extrait des premières valeurs de la bande 1 :
[[-99999. -99999. -99999. -99999. -99999.]
 [-99999. -99999. -99999. -99999. -99999.]
 [-99999. -99999. -99999. -99999. -99999.]
 [-99999. -99999. -99999. -99999. -99999.]
 [-99999. -99999. -99999. -99999. -99999.]]
